In [1]:
import os
import cv2
import albumentations as A
from glob import glob
from tqdm import tqdm

IMG_DIR = 'D:/MTU_Satelite_Project/Yaman_Dataset/Yolo11/Re_annotated_and_combined_images_no_aug/train/images'
LBL_DIR = 'D:/MTU_Satelite_Project/Yaman_Dataset/Yolo11/Re_annotated_and_combined_images_no_aug/train/labels'
OUT_IMG_DIR = 'D:/MTU_Satelite_Project/Yaman_Dataset/Yolo11/Re_annotated_and_combined_images_aug/images/train_aug'
OUT_LBL_DIR = 'D:/MTU_Satelite_Project/Yaman_Dataset/Yolo11/Re_annotated_and_combined_images_aug/labels/train_aug'
N_AUG_PER_IMAGE = 3  

os.makedirs(OUT_IMG_DIR, exist_ok=True)
os.makedirs(OUT_LBL_DIR, exist_ok=True)

transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=30, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.HueSaturationValue(p=0.3),
    A.GaussianBlur(p=0.2),
    A.GaussNoise(p=0.2),
    A.RandomScale(scale_limit=0.2, p=0.5),
], bbox_params=A.BboxParams(format='yolo', label_fields=['category_ids']))

image_paths = glob(os.path.join(IMG_DIR, '*.jpg')) 

for img_path in tqdm(image_paths):
    img = cv2.imread(img_path)
    height, width, _ = img.shape
    base_name = os.path.splitext(os.path.basename(img_path))[0]
    label_path = os.path.join(LBL_DIR, f"{base_name}.txt")

    if not os.path.exists(label_path):
        continue

    with open(label_path, 'r') as f:
        annotations = f.readlines()

    bboxes = []
    class_ids = []
    for ann in annotations:
        cls, x, y, w, h = map(float, ann.strip().split())
        bboxes.append([x, y, w, h])
        class_ids.append(int(cls))

    for i in range(N_AUG_PER_IMAGE):
        try:
            augmented = transform(image=img, bboxes=bboxes, category_ids=class_ids)
        except Exception:
            continue  

        aug_img = augmented['image']
        aug_bboxes = augmented['bboxes']
        aug_classes = augmented['category_ids']

        aug_img_name = f"{base_name}_aug{i}.jpg"
        cv2.imwrite(os.path.join(OUT_IMG_DIR, aug_img_name), aug_img)

        with open(os.path.join(OUT_LBL_DIR, f"{base_name}_aug{i}.txt"), 'w') as f:
            for cls, bbox in zip(aug_classes, aug_bboxes):
                x, y, w, h = bbox
                f.write(f"{cls} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")


  0%|          | 0/300 [00:00<?, ?it/s]


ValueError: too many values to unpack (expected 5)